In [1]:
from quant_met import hamiltonians, utils
import numpy as np
import pandas as pd
import time
from scipy import optimize

In [2]:
lattice_constant = np.sqrt(3)

all_K_points = 4 * np.pi / (3 * lattice_constant) * np.array([
    (np.sin(i * np.pi / 6), np.cos(i * np.pi / 6)) for i in [1, 3, 5, 7, 9, 11]]
)

BZ_grid = utils.generate_uniform_grid(20, 20, all_K_points[1], all_K_points[5], origin=np.array([0, 0]))

In [3]:
U_range = np.linspace(start=0.01, stop=5, num=10)

#for V in [0.1, 1, 2]:
for V in [0]:
    for mu in [0, -0.5, -1, -1.5]:
        gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
        gap_size_vs_U['U'] = U_range

        for i, U in enumerate(U_range):
            egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U, U_x=U)

            start = time.time()
            solution = optimize.brute(
                func=hamiltonians.free_energy,
                args=(
                    egx_h,
                    BZ_grid,
                    None,
                ),
                ranges=[(0, 1.5) for _ in range(egx_h.number_of_bands)],
                Ns=20,
                workers=-1,
                finish=optimize.fmin,
                #full_output=True,
            )
            print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
            end = time.time()
            print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
            gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution
        
        gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_U_uniform/V_{V}_mu_{mu}.csv')

V = 0, mu = 0, U = 0.01, solution: [0. 0. 0.]
Time taken to solve the gap equation: 87.18 seconds
V = 0, mu = 0, U = 0.5644444444444445, solution: [0.00262899 0.00263456 0.28114144]
Time taken to solve the gap equation: 92.12 seconds
V = 0, mu = 0, U = 1.118888888888889, solution: [0.00677587 0.00673106 0.55889983]
Time taken to solve the gap equation: 118.18 seconds
V = 0, mu = 0, U = 1.6733333333333336, solution: [0.01453368 0.01454073 0.83625937]
Time taken to solve the gap equation: 114.35 seconds
V = 0, mu = 0, U = 2.227777777777778, solution: [0.03437424 0.03433357 1.11359498]
Time taken to solve the gap equation: 95.34 seconds
V = 0, mu = 0, U = 2.7822222222222224, solution: [0.24272968 0.24272175 1.39087624]
Time taken to solve the gap equation: 139.95 seconds
V = 0, mu = 0, U = 3.336666666666667, solution: [0.90221311 0.90225485 1.66814985]
Time taken to solve the gap equation: 100.19 seconds
V = 0, mu = 0, U = 3.8911111111111114, solution: [1.31816639 1.31818019 1.94543479]
T

KeyboardInterrupt: 

In [13]:
U_range = np.linspace(start=0.01, stop=5, num=10)

U_gr = 1

for V in [0.1, 1, 2]:
    for mu in [0, -0.5, -1, -1.5]:
        gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
        gap_size_vs_U['U'] = U_range

        for i, U in enumerate(U_range):
            egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U_gr, U_x=U)

            start = time.time()
            solution = optimize.brute(
                func=hamiltonians.free_energy,
                args=(
                    egx_h,
                    BZ_grid,
                    None,
                ),
                ranges=[(0, 2) for _ in range(egx_h.number_of_bands)],
                Ns=20,
                workers=-1,
                finish=optimize.fmin,
                #full_output=True,
            )
            print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
            end = time.time()
            print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
            gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution

        gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_UX/V_{V}_mu_{mu}.csv')

mu = 0, U = 0.01, solution: [7.39694192e-04 5.12763546e-03 2.21835353e-05]
Time taken to solve the gap equation: 42.96 seconds
mu = 0, U = 0.5644444444444445, solution: [0.00597997 0.00751461 0.27917164]
Time taken to solve the gap equation: 45.81 seconds
mu = 0, U = 1.118888888888889, solution: [0.00659303 0.00711829 0.556684  ]
Time taken to solve the gap equation: 44.76 seconds
mu = 0, U = 1.6733333333333336, solution: [0.00660852 0.00690225 0.83412816]
Time taken to solve the gap equation: 43.37 seconds
mu = 0, U = 2.227777777777778, solution: [0.00661346 0.00674641 1.11145268]
Time taken to solve the gap equation: 47.69 seconds
mu = 0, U = 2.7822222222222224, solution: [0.00653957 0.00657567 1.38876919]
Time taken to solve the gap equation: 43.46 seconds
mu = 0, U = 3.336666666666667, solution: [0.00649126 0.00649987 1.66613379]
Time taken to solve the gap equation: 43.75 seconds
mu = 0, U = 3.8911111111111114, solution: [0.0072137  0.00519786 1.94343995]
Time taken to solve the g

In [15]:
U_range = np.linspace(start=0.01, stop=5, num=10)

for mu in [0, -0.5, -1, -1.5]:
    gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1'], index=range(len(U_range)))
    gap_size_vs_U['U'] = U_range

    for i, U in enumerate(U_range):
        graphene_h = hamiltonians.GrapheneHamiltonian(t_nn=1, a=lattice_constant, mu=mu, coulomb_gr=U)

        start = time.time()
        solution = optimize.brute(
            func=hamiltonians.free_energy,
            args=(
                graphene_h,
                BZ_grid,
                None,
            ),
            ranges=[(0, 2.5) for _ in range(graphene_h.number_of_bands)],
            Ns=20,
            workers=-1,
            finish=optimize.fmin,
            #full_output=True,
        )
        print(f'mu = {mu}, U = {U}, solution: {solution}')
        end = time.time()
        print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
        gap_size_vs_U.loc[i, ['delta_0', 'delta_1']] = solution

    gap_size_vs_U.to_csv(f'gap_plots_data/graphene/gap_size_vs_U/V_{V}_mu_{mu}.csv')

mu = 0, U = 0.01, solution: [0. 0.]
Time taken to solve the gap equation: 3.54 seconds
mu = 0, U = 0.5644444444444445, solution: [0.00262491 0.00263412]
Time taken to solve the gap equation: 4.30 seconds
mu = 0, U = 1.118888888888889, solution: [0.00675928 0.00680127]
Time taken to solve the gap equation: 4.28 seconds
mu = 0, U = 1.6733333333333336, solution: [0.01455579 0.01452332]
Time taken to solve the gap equation: 4.87 seconds
mu = 0, U = 2.227777777777778, solution: [0.03437146 0.03434924]
Time taken to solve the gap equation: 4.83 seconds
mu = 0, U = 2.7822222222222224, solution: [0.24274814 0.24271877]
Time taken to solve the gap equation: 4.59 seconds
mu = 0, U = 3.336666666666667, solution: [0.90228149 0.90230084]
Time taken to solve the gap equation: 4.80 seconds
mu = 0, U = 3.8911111111111114, solution: [1.3181425  1.31816455]
Time taken to solve the gap equation: 4.74 seconds
mu = 0, U = 4.445555555555556, solution: [1.66900804 1.66898575]
Time taken to solve the gap equa